**束搜索（Beam Search）**

一种近似的全局最优序列解码算法，每一步不只保留概率最高的一个候选（像贪心那样），而是保留前 B 个候选（束宽 beam size），并在下一步同时扩展它们，最后从所有完整候选里选得分最高的序列。

- 贪心（Greedy）：每步只取当前概率最高的 token，快，但容易走入局部最优；
- 采样（Temperature / Top-k / Top-p）：按分布抽样，多样性强，适合创作；可控性和稳定性较差；
- 束搜索（Beam）：每步保留多个最优候选，稳定、趋向高似然，但可能牺牲创意、多样性，且计算量随束宽上升。

核心原理如下
- 记序列 $y_{1:L}$ 的得分为对数似然的和 $\text{score}=\sum_{t=1}^{L}\log p(y_t\mid y_{<t},x)$
- 为避免“偏爱短句”，常用长度惩罚 / 归一化（GNMT 常用形式）
$$
\text{LP}(L)=\frac{(5+L)^\alpha}{(5+1)^\alpha},\quad
\text{final\_score}=\frac{\sum \log p}{\text{LP}(L)},\ \alpha\in[0,1]
$$
- 过程：维护大小为 B 的“束”，逐步扩展到所有词，再按（累计对数似然 + 惩罚）打分，剪枝为新的前 B 个；遇到 EOS 就放入“完成池”，直到达到最大长度或完成池数量满足要求。

一些工程技巧
- 长度惩罚 $\alpha\approx0.6$（经验值，可调）；
- 早停 / 多重早停：一旦完成池里已有的最优候选分数已不可能被未完成束超越，就停止；
- 多样化束搜索（Diverse Beam Search）：在不同束之间加入去重 / 分组惩罚，提升多样性；
- 受约束束搜索（Constrained / DFA / Regex Beam）：仅允许扩展到满足约束的 token（适合生成 JSON、SQL、函数调用参数等）；

如下先实现一个最朴素的 LM。

In [4]:
import math
from collections import namedtuple

EOS = "<eos>"
BeamItem = namedtuple("BeamItem", ["tokens", "logprob_sum", "finished"])
toy_table = {
    "<bos>": {"我": 0.6, "今": 0.3, "这": 0.1},
    "我": {"爱": 0.7, "要": 0.2, EOS: 0.1},
    "今": {"天": 0.8, EOS: 0.2},
    "这": {"个": 0.6, "是": 0.3, EOS: 0.1},
    "爱": {"NLP": 0.5, "你": 0.4, EOS: 0.1},
    "要": {"去": 0.7, EOS: 0.3},
    "天": {"很": 0.7, "下雨": 0.2, EOS: 0.1},
    "个": {"例子": 0.9, EOS: 0.1},
    "是": {"束搜索": 0.8, EOS: 0.2},
    "NLP": {EOS: 1.0},
    "你": {EOS: 1.0},
    "去": {"学习": 0.8, EOS: 0.2},
    "很": {"好": 0.9, EOS: 0.1},
    "下雨": {EOS: 1.0},
    "例子": {EOS: 1.0},
    "束搜索": {EOS: 1.0},
    "学习": {"束搜索": 0.6, "NLP": 0.3, EOS: 0.1},
    "好": {EOS: 1.0}
}

In [6]:
def toy_next_logprobs(prefix_tokens):
    last = prefix_tokens[-1] if prefix_tokens else "<bos>"
    probs = toy_table.get(last, {EOS: 1.0})
    return {tok: math.log(p) for tok, p in probs.items()}

def length_penalty(length, alpha=0.6):
    return ((5 + length) ** alpha) / ((5 + 1) ** alpha)

def beam_search(next_logprob_fn,
                beam_size=3,
                max_len=10,
                num_return_sequences=3,
                alpha=0.6):

    beams = [BeamItem(tokens=[], logprob_sum=0.0, finished=False)]
    finished = []

    for _ in range(max_len):
        candidates = []
        for b in beams:
            if b.finished:
                candidates.append(b)
                continue
            logprobs = next_logprob_fn(b.tokens if b.tokens else [])
            for tok, lp in logprobs.items():
                new_tokens = b.tokens + [tok]
                new_finished = (tok == EOS)
                new_logprob_sum = b.logprob_sum + lp
                candidates.append(BeamItem(new_tokens, new_logprob_sum, new_finished))

        def scored(item):
            L = len(item.tokens) if not item.finished else max(1, len(item.tokens)-1)
            return item.logprob_sum / length_penalty(L, alpha)

        candidates.sort(key=scored, reverse=True)

        new_beams = []
        for c in candidates:
            if len(new_beams) < beam_size:
                if c.finished:
                    finished.append(c)
                else:
                    new_beams.append(c)
        beams = new_beams

        # early stopping if no beams
        if not beams:
            break

    # if no finished, use current beams
    pool = finished if finished else beams
    pool.sort(key=lambda x: x.logprob_sum / length_penalty(
        max(1, len(x.tokens)-(1 if x.finished else 0)), alpha), reverse=True)

    outs = []
    for item in pool[:num_return_sequences]:
        toks = item.tokens[:-1] if item.finished and item.tokens and item.tokens[-1]==EOS else item.tokens
        outs.append((" ".join(toks), item.logprob_sum))
    return outs

In [8]:
results = beam_search(toy_next_logprobs, beam_size=3, max_len=10, num_return_sequences=3, alpha=0.6)
for i, (txt, lp) in enumerate(results, 1):
    print(f"[{i}] {txt} (sum logprob={lp:.3f})")

[1] 我 爱 NLP (sum logprob=-1.561)
[2] 今 天 很 好 (sum logprob=-1.889)
[3] 我 爱 你 (sum logprob=-1.784)
